### Treinando ALS - Tentativa para Rating Implicito

Vamos tentar utilizar um modelo de ALS dado que não temos feedback explicito das nossas recomendações e estaos trabalhando com um dataset relativamente grande o que pode nos ajudar no treinamento. 

In [1]:
# Libs:
import polars as pl
import numpy as np
from implicit.als import AlternatingLeastSquares
from implicit.evaluation import precision_at_k, mean_average_precision_at_k
from sklearn.model_selection import train_test_split
import scipy.sparse as sparse
import os

C:\Users\lucaa\Desktop\Pastas Gerais\Pós Tech Data Analytics\Fase 5\Datathon\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Não é recomendado rsrs
import warnings
warnings.filterwarnings("ignore")

### Pré Processamento de Dados

In [3]:
# Looping os conj. de treino:
path = "C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\files\\treino\\"
df_full = pl.DataFrame()
for file in os.listdir(path):
    if file.__contains__("csv"):
        df = pl.read_csv(f"{path}{file}", ignore_errors = True, schema = {
            'userId': pl.String,
            'userType': pl.String,
            'historySize': pl.String,
            'history': pl.String,
            'timestampHistory': pl.String,
            'numberOfClicksHistory': pl.String,
            'timeOnPageHistory': pl.String,
            'scrollPercentageHistory': pl.String,
            'pageVisitsCountHistory': pl.String,
            'timestampHistory_new': pl.String
        }, truncate_ragged_lines=True)
        print(f"Arquivo {file.replace('.csv', '')}: shape {df.shape}")
        
        # Concat dfs:
        df_full = pl.concat([df_full, df])
        del df

print(f"Full df: {df_full.shape}")

Arquivo treino_parte1: shape (100000, 10)
Arquivo treino_parte2: shape (100000, 10)
Arquivo treino_parte3: shape (100000, 10)
Arquivo treino_parte4: shape (100000, 10)
Arquivo treino_parte5: shape (100000, 10)
Arquivo treino_parte6: shape (77942, 10)
Full df: (577942, 10)


In [4]:
df_full.head()

userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
str,str,str,str,str,str,str,str,str,str
"""f98d1132f60d46883ce49583257104…","""Non-Logged""","""3""","""c8aab885-433d-4e46-8066-479f40…","""1657146417045, 1657146605778, …","""76, 38, 41""","""20380, 21184, 35438""","""50.3, 18.18, 16.46""","""2, 1, 1""","""1657146417045, 1657146605778, …"
"""2c1080975e257ed630e26679edbe4d…","""Non-Logged""","""60""","""3325b5a1-979a-4cb3-82b6-63905c…","""1656684240278, 1656761266729, …","""7, 80, 2, 1, 7, 62, 26, 44, 4,…","""6049, 210489, 8672, 10000, 300…","""25.35, 45.66, 35.3, 28.05, 36.…","""1, 1, 1, 1, 1, 1, 1, 1, 1, 2, …","""1656684240278, 1656761266729, …"
"""0adffd7450d3b9840d8c6215f0569a…","""Logged""","""107""","""04756569-593e-4133-a95a-83d35d…","""1656678946256, 1656701076495, …","""0, 0, 0, 0, 0, 44, 0, 0, 2, 1,…","""311274, 140000, 32515, 157018,…","""67.58, 47.22, 41.52, 63.09, 51…","""1, 1, 1, 1, 1, 1, 1, 1, 2, 1, …","""1656678946256, 1656701076495, …"
"""c1e8d644329a78ea1f994292db624c…","""Non-Logged""","""56""","""1f2b9c2f-a2d2-4192-b009-09065d…","""1658333312180, 1658404553818, …","""8, 0, 0, 0, 0, 0, 1, 0, 0, 1, …","""182696, 91925, 30000, 273655, …","""58.26, 72.66, 22.57, 59.89, 40…","""1, 1, 1, 1, 1, 1, 1, 1, 1, 1, …","""1658333312180, 1658404553818, …"
"""e777d1f31d4d955b63d60acc13df33…","""Non-Logged""","""4""","""bebdeb3e-1699-43e0-a1b8-989f5a…","""1658766608801, 1658766608801, …","""579, 579, 7, 2""","""801396, 801396, 10000, 10000""","""78.74, 78.74, 16.71, 9.34""","""7, 7, 1, 1""","""1658766608801, 1658766608801, …"


In [21]:
# Convertendo as infos para listas para facilitar posteriormente: 
df_full = (
    df_full
    .with_columns([
        pl.col("history").str.replace(r"\s+", "").str.split(",").alias("history_items"),
        pl.col("timestampHistory").str.replace(r"\s+", "").str.split(",").alias("ts_items"),
        pl.col("numberOfClicksHistory").str.replace(r"\s+", "").str.split(",").map_elements(lambda x: [int(i) for i in x]).alias("clicks"),
        pl.col("timeOnPageHistory").str.replace(r"\s+", "").str.split(",").map_elements(lambda x: [int(i) for i in x]).alias("timeOnPage"),
        pl.col("scrollPercentageHistory").str.replace(r"\s+", "").str.split(",").map_elements(lambda x: [float(i) for i in x]).alias("scrollPercentage")
    ])
)

In [22]:
df_full.head(2)

userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new,history_items,clicks,timeOnPage,scrollPercentage,ts_items
str,str,str,str,str,str,str,str,str,str,list[str],list[i64],list[i64],list[f64],list[str]
"""f98d1132f60d46883ce49583257104…","""Non-Logged""","""3""","""c8aab885-433d-4e46-8066-479f40…","""1657146417045, 1657146605778, …","""76, 38, 41""","""20380, 21184, 35438""","""50.3, 18.18, 16.46""","""2, 1, 1""","""1657146417045, 1657146605778, …","[""c8aab885-433d-4e46-8066-479f40ba7fb2"", ""68d2039c-c9aa-456c-ac33-9b2e8677fba7"", "" 13e423ce-1d69-4c78-bc18-e8c8f7271964""]","[76, 38, 41]","[20380, 21184, 35438]","[50.3, 18.18, 16.46]","[""1657146417045"", ""1657146605778"", "" 1657146698738""]"
"""2c1080975e257ed630e26679edbe4d…","""Non-Logged""","""60""","""3325b5a1-979a-4cb3-82b6-63905c…","""1656684240278, 1656761266729, …","""7, 80, 2, 1, 7, 62, 26, 44, 4,…","""6049, 210489, 8672, 10000, 300…","""25.35, 45.66, 35.3, 28.05, 36.…","""1, 1, 1, 1, 1, 1, 1, 1, 1, 2, …","""1656684240278, 1656761266729, …","[""3325b5a1-979a-4cb3-82b6-63905c9edbe8"", ""fe856057-f97d-419f-ab1c-97c5c3e0719c"", … "" 64e763a0-e086-4c41-b578-e72041d3da7c""]","[7, 80, … 5]","[6049, 210489, … 14231]","[25.35, 45.66, … 25.99]","[""1656684240278"", ""1656761266729"", … "" 1660518708052""]"


In [7]:
# Criando os objetos de user-item:
user_item_inter = []
for _row in df_full.iter_rows(named=True):
    user_id = _row['userId']
    for idx, item_id in enumerate(_row['history_items']):
        if idx < len(_row['clicks']) and idx < len(_row['timeOnPage']) and idx < len(_row['scrollPercentage']):
            user_item_inter.append({
                'userId': user_id,
                'itemId': item_id,
                'clicks': _row['clicks'][idx],
                'timeOnPage': _row['timeOnPage'][idx],
                'scrollPercentage': _row['scrollPercentage'][idx]
            })

In [8]:
len(user_item_inter)

8123951

In [9]:
user_item_inter[0:4]

[{'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': 'c8aab885-433d-4e46-8066-479f40ba7fb2',
  'clicks': 76,
  'timeOnPage': 20380,
  'scrollPercentage': 50.3},
 {'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': '68d2039c-c9aa-456c-ac33-9b2e8677fba7',
  'clicks': 38,
  'timeOnPage': 21184,
  'scrollPercentage': 18.18},
 {'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': ' 13e423ce-1d69-4c78-bc18-e8c8f7271964',
  'clicks': 41,
  'timeOnPage': 35438,
  'scrollPercentage': 16.46},
 {'userId': '2c1080975e257ed630e26679edbe4d5c850c65f3e09f655798b0bba9b42f2110',
  'itemId': '3325b5a1-979a-4cb3-82b6-63905c9edbe8',
  'clicks': 7,
  'timeOnPage': 6049,
  'scrollPercentage': 25.35}]

In [10]:
# Convertendo tudo para uma df:
df_user_item_inter = pl.DataFrame(user_item_inter, schema = {
    "userId": pl.String,
    "itemId": pl.String,
    "clicks": pl.Int32,
    "timeOnPage": pl.Int32,
    "scrollPercentage": pl.Float32,
    "rating": pl.Float32
})

In [11]:
df_user_item_inter.head(3)

userId,itemId,clicks,timeOnPage,scrollPercentage,rating
str,str,i32,i32,f32,f32
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",76,20380,50.299999,null
"""f98d1132f60d46883ce49583257104…","""68d2039c-c9aa-456c-ac33-9b2e86…",38,21184,18.18,null
"""f98d1132f60d46883ce49583257104…",""" 13e423ce-1d69-4c78-bc18-e8c8f…",41,35438,16.459999,null


Para podemos utilizar no modelo vamos criar uma euristica implicita para avaliar as recomendações e os itens que foram "avaliados" pelos usuários. Uma das formas de fazermos isso seria avaliar através das outras variáveis que servem como um medidor da qualidade da recomendação e daquela interação do usuário com a matério. Vamos criar um rating que leva em considerações os clicks na página, o tempo em que o usuário ficou nela e quanto "scrollou". Com essa combinação temos uma ideia se a interação foi positiva ou não e podemos recomendar matérias com interações de scores mais altos para os próximos usuários.

O Score será definido pela fórmula abaixo e pesos diferentes (foram definidos a priori pelo que consideramos interessante):

$$rating = clicks * 0.3 + timeOnPage * 0.4 + scrollPercentage * 0.3$$

In [12]:
# Criando os score rating:
df_user_item_inter = (
    df_user_item_inter
    .with_columns([
        (pl.col("clicks") * 0.3 + 
         pl.col("timeOnPage") * 0.0001 * 0.4 + 
         pl.col("scrollPercentage") * 0.3).alias("rating"),
        pl.col("itemId").str.strip_chars().alias("itemId")
    ])
)

In [13]:
df_user_item_inter.head(3)

userId,itemId,clicks,timeOnPage,scrollPercentage,rating
str,str,i32,i32,f32,f32
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",76,20380,50.299999,38.7052
"""f98d1132f60d46883ce49583257104…","""68d2039c-c9aa-456c-ac33-9b2e86…",38,21184,18.18,17.701361
"""f98d1132f60d46883ce49583257104…","""13e423ce-1d69-4c78-bc18-e8c8f7…",41,35438,16.459999,18.655519


In [14]:
user_id_map = {id: i for i, id in enumerate(df_user_item_inter["userId"].unique())}

In [16]:
item_id_map = {id: i for i, id in enumerate(df_user_item_inter["itemId"].unique())}

In [93]:
n_users, n_items = len(user_id_map), len(item_id_map)

In [137]:
# Add os indices dos users e items a df
df_user_item_inter = (
    df_user_item_inter
    .with_columns([
        pl.col("userId").map_elements(lambda x: user_id_map[x]).alias("user_idx"),
        pl.col("itemId").map_elements(lambda x: item_id_map[x]).alias("item_idx")
    ])
)

In [138]:
df_user_item_inter.head(3)

userId,itemId,clicks,timeOnPage,scrollPercentage,rating,user_idx,item_idx
str,str,i32,i32,f32,f32,i64,i64
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",76,20380,50.299999,38.7052,280033,104232
"""f98d1132f60d46883ce49583257104…","""68d2039c-c9aa-456c-ac33-9b2e86…",38,21184,18.18,17.701361,280033,115818
"""f98d1132f60d46883ce49583257104…","""13e423ce-1d69-4c78-bc18-e8c8f7…",41,35438,16.459999,18.655519,280033,47175


Agora começa o treinamento do modelo com as variáveis que montamos, porém precisamos criar alguns objetos como uma patriz sparsa pra poder alimentar o modelo no treinamento.

In [144]:
# ALS:
model = AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=15,
    calculate_training_loss=True
)

In [145]:
# Splitando pra treino:
df_train, df_test = train_test_split(df_user_item_inter, test_size = 0.2, random_state = 23)

In [149]:
# Matriz para treino já em CSR pra eficiencia:
m_train = sparse.coo_matrix((df_train['rating'].to_numpy(), (df_train['user_idx'].to_numpy(), df_train['item_idx'].to_numpy())),shape=(n_users, n_items)).tocsr()

In [150]:
m_train

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 6499160 stored elements and shape (577942, 255603)>

In [151]:
# Fit
model.fit(m_train)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:30<00:00,  2.05s/it, loss=0.000431]


In [152]:
# Matriz para teste já em CSR pra eficiencia:
m_test = sparse.coo_matrix((df_test['rating'].to_numpy(), (df_test['user_idx'].to_numpy(), df_test['item_idx'].to_numpy())),shape=(n_users, n_items)).tocsr()

In [153]:
m_test

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 1615045 stored elements and shape (577942, 255603)>

In [112]:
# Custom evaluation instead of using implicit's built-in evaluation
def precision_at_k(model, train_matrix, test_matrix, K=10):
    """
    Calculate precision@k for all users
    """
    # Number of users
    n_users = train_matrix.shape[0]
    precisions = []
    
    for user_id in range(n_users):
        # Get the items this user rated in the test set
        test_items = set(test_matrix[user_id].indices)
        if len(test_items) == 0:
            continue
            
        # Get recommendations for this user
        recommended_items = model.recommend(user_id, train_matrix[user_id], N=K)[0]
        
        # Calculate precision
        n_relevant = sum(1 for item in recommended_items if item in test_items)
        precisions.append(n_relevant / K)
        
    return np.mean(precisions) if precisions else 0.0

def mean_average_precision_at_k(model, train_matrix, test_matrix, K=10):
    """
    Calculate MAP@k for all users
    """
    # Number of users
    n_users = train_matrix.shape[0]
    avg_precisions = []
    
    for user_id in range(n_users):
        # Get the items this user rated in the test set
        test_items = set(test_matrix[user_id].indices)
        if len(test_items) == 0:
            continue
            
        # Get recommendations for this user
        recommended_items = model.recommend(user_id, train_matrix[user_id], N=K)[0]
        
        # Calculate average precision
        relevant_items = 0
        sum_precisions = 0
        
        for i, item in enumerate(recommended_items):
            if item in test_items:
                relevant_items += 1
                precision_at_i = relevant_items / (i + 1)
                sum_precisions += precision_at_i
                
        if relevant_items > 0:
            avg_precisions.append(sum_precisions / min(relevant_items, K))
        
    return np.mean(avg_precisions) if avg_precisions else 0.0

In [160]:
# Evaluate the model with custom evaluation functions
p_at_k = precision_at_k(model, m_train, m_test, K = 5)
map_at_k = mean_average_precision_at_k(model, m_train, m_test, K = 5)

print(f"Precision@10: {p_at_k}")
print(f"MAP@10: {map_at_k}")

Precision@10: 0.00594788686701861
MAP@10: 0.477648031358122


In [162]:
# Generate recommendations for a specific user
def get_recommendations(user_id, model, user_id_map, item_id_map, user_items, n=10):
    # Map user_id to user_idx
    if user_id not in user_id_map:
        print(f"User {user_id} not found in training data")
        return []
    
    user_idx = user_id_map[user_id]
    # Get recommendations
    recommendations = model.recommend(user_idx, user_items[user_idx], N=n)
    
    # Map item_idx back to item_id
    reverse_item_map = {v: k for k, v in item_id_map.items()}
    return [(reverse_item_map[item_idx], score) for item_idx, score in zip(recommendations[0], recommendations[1])]

# Example: Get recommendations for the first user
first_user_id = user_ids[0]
user_recommendations = get_recommendations(first_user_id, model, user_id_map, item_id_map, m_train)
print(f"Recommendations for user {first_user_id}:")
for item_id, score in user_recommendations:
    print(f"Item: {item_id}, Score: {score:.4f}")

Recommendations for user a4a6fb7be6d718211d74eb1484bf79e841b1f660fb76b5f43de967592f8f3c65:
Item: 1f32787b-de2b-49be-8c20-ddaeae34cc22, Score: 0.1123
Item: f6b5d170-48b9-4f8e-88d4-c84b6668f3bd, Score: 0.0773
Item: 97c00ceb-20e2-455e-b706-9a720e9b32d2, Score: 0.0756
Item: d173164f-dcfc-4e32-8006-64ab44a57c66, Score: 0.0754
Item: 1db5507f-ba78-4c81-96f1-36a4874fca87, Score: 0.0731
Item: b442d94e-f0c5-4fc1-9406-a4b64b595d71, Score: 0.0702
Item: 2e3300c6-3a18-4908-b9a9-a955cfcdcfcc, Score: 0.0659
Item: b7f30490-0b67-4ccc-b032-1eeeb179b464, Score: 0.0633
Item: 3901a861-007c-45ad-94ac-c635d8663696, Score: 0.0603
Item: 33b8607c-f4aa-4379-a18d-f7dddd2dd52e, Score: 0.0596


In [67]:
df_val = pl.read_csv("C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\validacao.csv", ignore_errors = True, schema = {
            'userId': pl.String,
            'userType': pl.String,
            'history': pl.String,
            'timestampHistory': pl.String
        }, truncate_ragged_lines=True)

In [68]:
df_val.head(3)

userId,userType,history,timestampHistory
str,str,str,str
"""e25fbee3a42d45a2914f9b061df338…","""Logged""","""['be89a7da-d9fa-49d4-9fdc-388c…","""[1660533136590 1660672113513]"""
"""d0afad7ea843d86597d822f0df1d39…","""Logged""","""['77901133-aee7-4f7b-afc0-6522…","""[1660556860253]"""
"""755062dd39a48809880cf363b04268…","""Logged""","""['857aa90f-a7ec-410d-ba82-dfa4…","""[1660561649242]"""


In [ ]:
# Generate recommendations for all users
recomm_val = {}
for user_id in df_val["userId"]:
    recomm_val[user_id] = get_recommendations(user_id, model, user_id_map, item_id_map, train_data)

In [100]:
cont = {}
for uuid in recomm_val:
    t = df_val.filter((pl.col("userId") == uuid)).select("history").item(0, 0)
    c = 0
    for r_i in recomm_val[uuid]:
        if t.__contains__(r_i[0]):
            c += 1
    cont[uuid] = c

In [104]:
sum(cont.values())

357

In [105]:
len(cont)

31291

In [159]:
357 / 31291

0.011409031350867661

In [163]:
# After training the model
import pickle

# Save the model and mappings
model_data = {
    'model': model,
    'user_id_map': user_id_map,
    'item_id_map': item_id_map,
    'reverse_user_map': {v: k for k, v in user_id_map.items()},
    'reverse_item_map': {v: k for k, v in item_id_map.items()}
}

with open('als_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("Model and mappings saved to 'als_model.pkl'")

Model and mappings saved to 'als_model.pkl'


In [165]:
df_val.select("userId").item(0, 0)

'e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9'

Nós temmos um modelo padrão 1 para incluirmos na API, porém podemos tentar melhorar um pouco ele, ainda mais pelos resultaods.
Relembrando, tivemos métricas de 

- Precision@10: 0.00594788686701861
- MAP@10: 0.477648031358122

Ou seja, nosso modelo não acerta tanto na recomendação mas as mesmas parecem ser relevantes.

#### Próximos passos

- O problema do Cold Start, dado que estamos usando feedback implicito e um modelo ALS não é um dos melhores jeitos de se lidar, dessa forma iremos fazer um ajuste para recomendar itens mais populares sempre que identificarmos um novo usuário na API.
- A parte da recência é mais complexa, um ajuste simples que iremos realizar nesse momento é deixar no conjunto de dados somente interações mais recentes, o que ajuda a lidar um pouco com a situação. 

In [18]:
df_full.head(1)

userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new,history_items,clicks,timeOnPage,scrollPercentage
str,str,str,str,str,str,str,str,str,str,list[str],list[i64],list[i64],list[f64]
"""f98d1132f60d46883ce49583257104…","""Non-Logged""","""3""","""c8aab885-433d-4e46-8066-479f40…","""1657146417045, 1657146605778, …","""76, 38, 41""","""20380, 21184, 35438""","""50.3, 18.18, 16.46""","""2, 1, 1""","""1657146417045, 1657146605778, …","[""c8aab885-433d-4e46-8066-479f40ba7fb2"", ""68d2039c-c9aa-456c-ac33-9b2e8677fba7"", "" 13e423ce-1d69-4c78-bc18-e8c8f7271964""]","[76, 38, 41]","[20380, 21184, 35438]","[50.3, 18.18, 16.46]"


In [28]:
from datetime import datetime

In [38]:
# Criando uma nova df de itens:
user_item_inter = []
for _row in df_full.iter_rows(named=True):
    user_id = _row['userId']
    for idx, item_id in enumerate(_row['history_items']):
        if idx < len(_row['clicks']) and idx < len(_row['timeOnPage']) and idx < len(_row['scrollPercentage']):
            user_item_inter.append({
                'userId': user_id,
                'itemId': item_id,
                'ts_inter': _row['ts_items'][idx],
                'clicks': _row['clicks'][idx],
                'timeOnPage': _row['timeOnPage'][idx],
                'scrollPercentage': _row['scrollPercentage'][idx]
            })

In [39]:
user_item_inter[0:4]

[{'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': 'c8aab885-433d-4e46-8066-479f40ba7fb2',
  'ts_inter': '1657146417045',
  'clicks': 76,
  'timeOnPage': 20380,
  'scrollPercentage': 50.3},
 {'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': '68d2039c-c9aa-456c-ac33-9b2e8677fba7',
  'ts_inter': '1657146605778',
  'clicks': 38,
  'timeOnPage': 21184,
  'scrollPercentage': 18.18},
 {'userId': 'f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069',
  'itemId': ' 13e423ce-1d69-4c78-bc18-e8c8f7271964',
  'ts_inter': ' 1657146698738',
  'clicks': 41,
  'timeOnPage': 35438,
  'scrollPercentage': 16.46},
 {'userId': '2c1080975e257ed630e26679edbe4d5c850c65f3e09f655798b0bba9b42f2110',
  'itemId': '3325b5a1-979a-4cb3-82b6-63905c9edbe8',
  'ts_inter': '1656684240278',
  'clicks': 7,
  'timeOnPage': 6049,
  'scrollPercentage': 25.35}]

In [55]:
# Nova df:
# Convertendo tudo para uma df:
df_user_item_inter_v2 = pl.DataFrame(user_item_inter, schema = {
    "userId": pl.String,
    "itemId": pl.String,
    "ts_inter": pl.String,
    "clicks": pl.Int32,
    "timeOnPage": pl.Int32,
    "scrollPercentage": pl.Float32,
    "rating": pl.Float32
})

In [56]:
df_user_item_inter_v2.head(2)

userId,itemId,ts_inter,clicks,timeOnPage,scrollPercentage,rating
str,str,str,i32,i32,f32,f32
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…","""1657146417045""",76,20380,50.299999,null
"""f98d1132f60d46883ce49583257104…","""68d2039c-c9aa-456c-ac33-9b2e86…","""1657146605778""",38,21184,18.18,null


In [57]:
# Criando os score rating:
df_user_item_inter_v2 = (
    df_user_item_inter_v2
    .with_columns([
        (pl.col("clicks") * 0.3 + 
         pl.col("timeOnPage") * 0.0001 * 0.4 + 
         pl.col("scrollPercentage") * 0.3).alias("rating"),
        pl.col("itemId").str.strip_chars().alias("itemId"),
        pl.col("ts_inter").str.strip_chars().cast(pl.Int64).cast(pl.Datetime("ms"))
    ])
)

In [58]:
df_user_item_inter_v2.head(3)

userId,itemId,ts_inter,clicks,timeOnPage,scrollPercentage,rating
str,str,datetime[ms],i32,i32,f32,f32
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",2022-07-06 22:26:57.045,76,20380,50.299999,38.7052
"""f98d1132f60d46883ce49583257104…","""68d2039c-c9aa-456c-ac33-9b2e86…",2022-07-06 22:30:05.778,38,21184,18.18,17.701361
"""f98d1132f60d46883ce49583257104…","""13e423ce-1d69-4c78-bc18-e8c8f7…",2022-07-06 22:31:38.738,41,35438,16.459999,18.655519


In [59]:
# Distribuicoes das datas:
(
    df_user_item_inter_v2.with_columns(
        pl.col("ts_inter").dt.year().alias("ano"),
        pl.col("ts_inter").dt.month().alias("mes")
    )
    .group_by(["ano", "mes"])
    .agg(pl.len().alias("count"))
    .sort(["ano", "mes"])  # Sort by year and month
)

ano,mes,count
i32,i8,u32
2022,7,5344389
2022,8,2779562


Só temos informações de duas safras então vamos ter que avaliar especificamente as matérias.

In [63]:
# Looping os conj. de treino:
path = "C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\data\\itens\\itens\\"
df_materias = pl.DataFrame()
for file in os.listdir(path):
    if file.__contains__("csv"):
        df = pl.read_csv(f"{path}{file}", ignore_errors = True, schema = {
            'page': pl.String,
            'url': pl.String,
            'issued': pl.String,
            'modified': pl.String,
            'title': pl.String,
            'body': pl.String,
            'caption': pl.String
        }, truncate_ragged_lines=True)
        print(f"Arquivo {file.replace('.csv', '')}: shape {df.shape}")
        
        # Concat dfs:
        df_materias = pl.concat([df_materias, df])
        del df

print(f"Full df: {df_materias.shape}")

Arquivo itens-parte1: shape (100000, 7)
Arquivo itens-parte2: shape (100000, 7)
Arquivo itens-parte3: shape (55603, 7)
Full df: (255603, 7)


In [64]:
df_materias.head(3)

page,url,issued,modified,title,body,caption
str,str,str,str,str,str,str
"""13db0ab1-eea2-4603-84c4-f40a87…","""http://g1.globo.com/am/amazona…","""2022-06-18 20:37:45+00:00""","""2023-04-15 00:02:08+00:00""","""Caso Bruno e Dom: 3º suspeito …","""Após audiência de custódia, a …","""Jeferson da Silva Lima foi esc…"
"""92907b73-5cd3-4184-8d8c-e206ae…","""http://g1.globo.com/pa/santare…","""2019-06-20 17:19:52+00:00""","""2023-06-16 20:19:15+00:00""","""Linguajar dos santarenos é dif…","""Vista aérea de Santarém Ádrio …","""As expressões santarenas não s…"
"""61e07f64-cddf-46f2-b50c-ea0a39…","""http://g1.globo.com/mundo/noti…","""2022-07-08 08:55:52+00:00""","""2023-04-15 04:25:39+00:00""","""Ex-premiê Shinzo Abe morre apó…","""Novo vídeo mostra que assassin…","""Ex-primeiro-ministro foi ating…"


In [76]:
# Tempo da materia pras safras de interações:
(
    df_materias.with_columns(
        pl.col("issued").str.to_datetime(format="%Y-%m-%d %H:%M:%S%z").dt.year().alias("ano_issued"),
        pl.col("modified").str.to_datetime(format="%Y-%m-%d %H:%M:%S%z").dt.year().alias("ano_mod")
    )
    .group_by(["ano_issued", "ano_mod"])
    .agg(pl.len().alias("count"))
    .sort(["count"], descending=True)
)

ano_issued,ano_mod,count
i32,i32,u32
2022,2022,100944
2021,2021,60421
2020,2020,33004
2019,2019,26015
2018,2018,20735
…,…,…
2015,2015,2
2015,2017,2
2014,2014,1


Vamos tirar todas as matérias publicadas pré 2022, até porque as matérias não parecem ser mdificadas muito depois.

In [77]:
# Df filtrada para a modelagem sem casos antigos:
df_materias_f = (
    df_materias
    .with_columns(pl.col("issued").str.to_datetime(format="%Y-%m-%d %H:%M:%S%z").dt.year().alias("ano_issued"))
    .filter((pl.col("ano_issued") == 2022))
)

In [80]:
list_materias_2022 = df_materias_f["page"].to_list()

In [84]:
# Dt Final Modelagem - Recencia:
df_user_item_inter_modelagem = df_user_item_inter_v2.filter(pl.col("itemId").is_in(list_materias_2022))

In [85]:
df_user_item_inter_modelagem.shape

(7462272, 7)

In [86]:
user_id_map = {id: i for i, id in enumerate(df_user_item_inter_modelagem["userId"].unique())}
item_id_map = {id: i for i, id in enumerate(df_user_item_inter_modelagem["itemId"].unique())}

In [87]:
df_user_item_inter_modelagem = (
    df_user_item_inter_modelagem
    .with_columns([
        pl.col("userId").map_elements(lambda x: user_id_map[x]).alias("user_idx"),
        pl.col("itemId").map_elements(lambda x: item_id_map[x]).alias("item_idx")
    ])
)

In [88]:
df_user_item_inter_modelagem.head(3)

userId,itemId,ts_inter,clicks,timeOnPage,scrollPercentage,rating,user_idx,item_idx
str,str,datetime[ms],i32,i32,f32,f32,i64,i64
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",2022-07-06 22:26:57.045,76,20380,50.299999,38.7052,204276,89205
"""f98d1132f60d46883ce49583257104…","""13e423ce-1d69-4c78-bc18-e8c8f7…",2022-07-06 22:31:38.738,41,35438,16.459999,18.655519,204276,47180
"""2c1080975e257ed630e26679edbe4d…","""3325b5a1-979a-4cb3-82b6-63905c…",2022-07-01 14:04:00.278,7,6049,25.35,9.94696,81606,98197


In [89]:
# ALS - versao atualizada:
model_v2 = AlternatingLeastSquares(
    factors=50,
    regularization=0.01,
    iterations=15,
    calculate_training_loss=True
)

In [103]:
# Splitando pra treino:
df_train_v2, df_test_v2 = train_test_split(df_user_item_inter_modelagem, test_size = 0.2, random_state = 23)

In [107]:
n_users_v2, n_items_v2 = df_user_item_inter_modelagem["userId"].n_unique(), df_user_item_inter_modelagem["itemId"].n_unique()

In [108]:
# Matriz para treino já em CSR pra eficiencia:
m_train_v2 = sparse.coo_matrix((df_train_v2['rating'].to_numpy(), (df_train_v2['user_idx'].to_numpy(), df_train_v2['item_idx'].to_numpy())),shape=(n_users_v2, n_items_v2)).tocsr()
# Matriz para teste já em CSR pra eficiencia:
m_test_v2 = sparse.coo_matrix((df_test_v2['rating'].to_numpy(), (df_test_v2['user_idx'].to_numpy(), df_test_v2['item_idx'].to_numpy())),shape=(n_users_v2, n_items_v2)).tocsr()

In [109]:
model_v2.fit(m_train_v2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:21<00:00,  1.43s/it, loss=0.0014]


In [113]:
# Avaliando o modelo 2:
p_at_k = precision_at_k(model_v2, m_train_v2, m_test_v2, K = 5)
map_at_k = mean_average_precision_at_k(model_v2, m_train_v2, m_test_v2, K = 5)

print("Performance Modelo V2 Ajustado")
print(f"Precision@10: {p_at_k}")
print(f"MAP@10: {map_at_k}")

Performance Modelo V2 Ajustado
Precision@10: 0.04659236179634374
MAP@10: 0.5523671494408061


Melhoramos nosso desempenho com essas poucas mudanças, perfeito.

#### Exportando modelo final:

In [114]:
# Matrix geral:
m_total = sparse.coo_matrix((df_user_item_inter_modelagem['rating'].to_numpy(), (df_user_item_inter_modelagem['user_idx'].to_numpy(), df_user_item_inter_modelagem['item_idx'].to_numpy())),shape=(n_users_v2, n_items_v2)).tocsr()

In [115]:
m_total

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 7462272 stored elements and shape (379954, 101120)>

In [116]:
model_v2.fit(m_total)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:27<00:00,  1.81s/it, loss=0.00163]


In [125]:
df_user_item_inter_modelagem.head(2)

userId,itemId,ts_inter,clicks,timeOnPage,scrollPercentage,rating,user_idx,item_idx
str,str,datetime[ms],i32,i32,f32,f32,i64,i64
"""f98d1132f60d46883ce49583257104…","""c8aab885-433d-4e46-8066-479f40…",2022-07-06 22:26:57.045,76,20380,50.299999,38.7052,204276,89205
"""f98d1132f60d46883ce49583257104…","""13e423ce-1d69-4c78-bc18-e8c8f7…",2022-07-06 22:31:38.738,41,35438,16.459999,18.655519,204276,47180


In [139]:
# Itens mais populares para novos usuarios:
lista_populares = (
    df_user_item_inter_modelagem
    .group_by(["itemId"])
    .agg(
        pl.len().alias("count"),
        pl.col("rating").mean().alias("avg_rating")
    )
    .sort(["count", "avg_rating"], descending=True)
    .head(100)
    .sort(["avg_rating"], descending=True)
    ["itemId"]
    .to_list()
)

In [140]:
model_data_final = {
    'model': model_v2,
    'user_id_map': user_id_map,
    'item_id_map': item_id_map,
    'items_popularity': lista_populares,
    'reverse_user_map': {v: k for k, v in user_id_map.items()},
    'reverse_item_map': {v: k for k, v in item_id_map.items()}
}

In [141]:
import pickle

In [142]:
with open('C:\\Users\\lucaa\\Desktop\\Pastas Gerais\\Pós Tech Data Analytics\\Fase 5\\Datathon\\model\\als_model_v2.pkl', 'wb') as f:
    pickle.dump(model_data_final, f)